# Optimizing Python Code

In [571]:
import cProfile
import random
import math
import numpy as np
import time
import multiprocessing as mp
import cython
import matplotlib.pyplot as plt
from scipy.special import k1e, kve
from scipy.integrate import quad
from PIL import Image

from ipywidgets import interact

%matplotlib inline
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## Introduction

Often times (especially in `Python`) one finds that their code just isn't fast enough and they would like to speed up their code. Say, they're running some sort of scan which is taking many hours to run. How might they reduce this time to something more managable, like several minutes? In these tutorials, we will investigate techniques to speed up code in python.

But before we dig in, we will need to set some ground rules:
1. Make sure your code is working **BEFORE** trying to optimize.
2. Profile your code to find where your code is the slowest.
3. Try using `NumPy` first.
4. Try other options

<div class="alert alert-warning">
==WARNING==
    
In regards to point #2:
   
In Donald Knuth's paper "Structured Programming With Go To Statements", he wrote: 
    
"Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered. We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil. Yet we should not pass up our opportunities in that critical 3%.
</div>

The take away from the above statements is: you should find where you code is taking the longest first before trying to optimize. You may spend a lot of time optimizing what you *think* is the slow part of your code only to realize when you're finished that your optimizations we're did very little. To discover where you should look to optimize, one should **profile** their code.

## Profiling

Profiling you code is very easy in python. One can simply import the `cProfile` module from the python standard library and run `cProfile.run(...)` where the `...` is the code you would like to profile.

To demonstrate how one might use `cProfile` to find slow parts of their code, let's revisit the Ising model. Recall that the Ising model consists of $N$ spins which interact with their nearest neighbors with an interaction strength $J$. If these spins have a magnetic moment, $\mu$, they also interact with external magentic fields $B$. The Hamiltonian for this system was given by:

\begin{align}
    H = -J\sum_{i=0}^{N-1}S_{i}S_{i+1} - \mu B\sum_{i=0}^{N-1}S_{i}
\end{align}

where $S_{i}=1,-1$ and $S_{N + i} \equiv S_{i}$ for periodic boundary conditions. When we investigating the thermodynamic properties Ising model, we used the Metropolis algorithm to relax the spin-chain to an equilibrium state. This algorithm worked applying the following steps:
1. Pick a random spin. Call it $S_{k}$
2. Compute the change in energy, $\Delta E$, needed to flip $S_{k}$
3. If $\Delta E$ < 0, flip the spin. If $\Delta E \geq 0$, we draw a random uniform number $r$. If $e^{-\beta\Delta E}\geq r$, then we flip the spin.
4. Repeat 1-3 many times.
5. Average over the last several microstates to determine the macro thermodynamic property in question.  

Let's implement steps 1-4. First, let's construct a function which computes the change in energy of the spin-chain if we flip spin $i$. For our first pass, we will compute the energy for the original state, then recompute the energy for the new state. We will first write a function to compute the energy:

In [481]:
def compute_energy(spin_chain, J, B, mu):
    """
    Compute the energy of the spin configuration.
    """
    n = len(spin_chain)
    energy = 0.0
    for i in range(n):
        # Compute the energy from spin-spin interactions
        spinspin = -J * spin_chain[i - 1] * spin_chain[i]
        # Compute the energy from spin-magnetic field interactions
        mag = -mu * B * spin_chain[i]
        energy += spinspin + mag
    return energy

Now, let's write a function to compute the change in energy if we perform a flip:

In [482]:
def compute_energy_change(i, spin_chain, J, B, mu):
    """
    Compute the energy of the spin configuration.
    """
    # Compute the energy of the state
    Ei = compute_energy(spin_chain, J, B, mu)
    # Compute the energy of the flipped state
    spin_chain[i] *= -1
    Ef = compute_energy(spin_chain, J, B, mu)
    # Compute the difference and restore state:
    dE = Ef - Ei
    spin_chain[i] *= -1
    return dE

Next, let's implement a function to update the state of our system using the metropolis algorithm:

In [487]:
def update_spin(spin_chain, J, B, mu, T):
    """
    Update a spin in the spin chain.
    """
    n = len(spin_chain)
    # Use `random` to choose an integer from 0, n-1
    i = random.randint(0, n-1)
    # Compute change in energy
    dE = compute_energy_change(i, spin_chain, J, B, mu)
    # Apply Metropolis
    if math.exp(-dE / T) <= random.random():
        spin_chain[i] *= -1

Now let's profile the `update_spin` function and see how things are going:

In [488]:
nspins = 1000
J = 1.0
B = 0.0
mu = 0.0
T = 1.0
spin_chain = [random.choice([-1, 1]) for _ in range(nspins)]

cProfile.run('[update_spin(spin_chain, J, B, mu, T) for _ in range(10000)]')

         140263 function calls in 4.518 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    20000    4.461    0.000    4.463    0.000 <ipython-input-481-4e573e4722c4>:1(compute_energy)
    10000    0.009    0.000    4.472    0.000 <ipython-input-482-0a6d95fc7e8e>:1(compute_energy_change)
    10000    0.014    0.000    4.513    0.000 <ipython-input-487-99df470a5c94>:1(update_spin)
        1    0.004    0.004    4.518    4.518 <string>:1(<listcomp>)
        1    0.000    0.000    4.518    4.518 <string>:1(<module>)
    10000    0.009    0.000    0.019    0.000 random.py:172(randrange)
    10000    0.004    0.000    0.023    0.000 random.py:216(randint)
    10000    0.007    0.000    0.010    0.000 random.py:222(_randbelow)
        1    0.000    0.000    4.518    4.518 {built-in method builtins.exec}
    30000    0.003    0.000    0.003    0.000 {built-in method builtins.len}
    10000    0.002    0.000    0.002    0.000 {buil

Here is what everything means:
- **ncalls**: for the number of calls.
- **tottime**: for the total time spent in the given function (and excluding time made in calls to sub-functions)
- **percall**: is the quotient of tottime divided by ncalls
- **cumtime**: is the cumulative time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions.
- **percall**: is the quotient of cumtime divided by primitive calls
- **filename:lineno(function)**: provides the respective data of each function

So we can see that 10000 calls to `update_spin` takes about 4.4 seconds. However, we see something interesting. Nearly ALL of that time is spent in `compute` energy (see `tottime`.) So, if we would like to optimize this function, its pretty obvious where to start! We should optimize `compute_energy`. But, if we think a bit harder, we only want compute energy to compute the change in energy of the system. Since only one spin changes at a time, we can bypass `compute_energy` all together and come up with a simpler way to compute the change in energy.

Looking more closely at the Hamiltonian, one finds that, if we flip spin $k$, the change in energy of the system is:

\begin{align}
    \Delta E = 2 S_{k} \left[J\left(S_{k+1} + S_{k-1}\right) + \mu B\right]
\end{align}

Thus, to determine the change in energy of the system, simply need to compute the above. Let's make a new version of our `compute_energy_change` function:

In [489]:
def compute_energy_change2(k, spin_chain, J, B, mu):
    if k == len(spin_chain) - 1:
        return 2 * spin_chain[k] * (J * (spin_chain[0] + spin_chain[k-1]) + mu * B)
    else:
        return 2 * spin_chain[k] * (J * (spin_chain[k+1] + spin_chain[k-1]) + mu * B)

As we should always do, let's make sure this new implementation agrees with the previous implemenation:

In [490]:
print(compute_energy_change(1, spin_chain, J, B, mu))
print(compute_energy_change2(1, spin_chain, J, B, mu))

-4.0
-4.0


Good! Now, let's make a new version of our update function which uses the new version of the `compute_energy_change` function:

In [493]:
def update_spin2(spin_chain, J, B, mu, T):
    """
    Update a spin in the spin chain.
    """
    n = len(spin_chain)
    # Use `random` to choose an integer from 0, n-1
    i = random.randint(0, n-1)
    # Compute change in energy
    dE = compute_energy_change2(i, spin_chain, J, B, mu)
    # Apply Metropolis
    if math.exp(-dE / T) <= random.random():
        spin_chain[i] *= -1

And let's give it a whirl:

In [494]:
nspins = 1000
J = 1.0
B = 0.0
mu = 0.0
T = 1.0
spin_chain = [random.choice([-1, 1]) for _ in range(nspins)]

cProfile.run('[update_spin2(spin_chain, J, B, mu, T) for _ in range(10000)]')

         110254 function calls in 0.047 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.006    0.000    0.007    0.000 <ipython-input-489-6d62445ad29f>:1(compute_energy_change2)
    10000    0.012    0.000    0.043    0.000 <ipython-input-493-e94d820e9b94>:1(update_spin2)
        1    0.004    0.004    0.047    0.047 <string>:1(<listcomp>)
        1    0.000    0.000    0.047    0.047 <string>:1(<module>)
    10000    0.008    0.000    0.017    0.000 random.py:172(randrange)
    10000    0.004    0.000    0.021    0.000 random.py:216(randint)
    10000    0.006    0.000    0.009    0.000 random.py:222(_randbelow)
        1    0.000    0.000    0.047    0.047 {built-in method builtins.exec}
    20000    0.002    0.000    0.002    0.000 {built-in method builtins.len}
    10000    0.001    0.000    0.001    0.000 {built-in method math.exp}
    10000    0.001    0.000    0.001    0.000 {method 'bit_length' of 'in

😮... This is about 100 times faster! This illustrates how we can pinpoint the slowest parts of our code and optimize to achieve faster code.

## `NumPy`

Often times when you have slow `Python` code in scientific computing its because you're not using enough `NumPy`! A lot of `NumPy` is written is pure `c`. That means that when you do operations on arrays, these operations are often being done in pure `c` behind the scenes. If we take adavantage of this, we can often achieve large speed ups.

To illustrate this, let's return to the Monte-carlo integration example from one of the PSETs. Remember that Monte-Carlo integration works by approximating an integral $I$ using:

\begin{align}
    I = \int_{\Omega}d^{d}\vec{x}f(\vec{x})\approx \frac{V}{N}\sum_{i=1}^{N}f(\vec{x}_{i})
\end{align}

where $\vec{x}_{i}$ is a random vector inside our region of integration $\Omega$ with volume $V$. For simplicity, we will take $\Omega$ to be a unit hyper-cube (so $V = 1$.) Let's write a function which will use MC integration to integrate some function over a unit hyper-cube.

In [496]:
def integrate_mc(f, ndim, npts):
    """
    Integrate a function over a hyper-cube using Monte-Carlo.
    
    Parameters
    ----------
    f: callable
        Function to integrate
    ndim: int
        Number of dimensions of the system
    npts: int
        Number of points to use in integration.
    """
    fsum = 0.0
    f2sum = 0.0
    for k in range(npts):
        # Generate a random vector
        pts = [random.random() for i in range(ndim)]
        # Evaluate the function
        fval = f(pts)
        fsum += fval
        f2sum += fval**2
    # Compute the integral estimate
    integral = fsum / npts
    avg2 = f2sum / npts
    std = math.sqrt(avg2 - integral**2)
    error = std / math.sqrt(npts)
    
    return integral, error

Let's use the same function from the PSET, i.e.:

\begin{align}
f(x) = \left(\sum_{i=1}^{10}x_{i}\right)^2
\end{align}

In [497]:
def f(x):
    fval = 0.0
    for i in range(len(x)):
        fval += x[i]
    return fval**2

Now let's time how long this function takes:

In [499]:
t0 = time.time()
npts = 5000000
integral, error = integrate_mc(f, 10, npts)
t1 = time.time()
print(f"integral = {integral}, error = {error}, time = {t1-t0}")

integral = 25.825734006092446, error = 0.004112657973875036, time = 10.836636781692505


Let's think about what we are doing in the above example. We are first taking `d=ndim` random number:
\begin{align}
    r_{1}, r_{2},\dots, r_{d}
\end{align}
Then, we are summing them:
\begin{align}
    r_{1}, r_{2},\dots, r_{d} \to r_{1} + r_{2}+ \cdots + r_{d}
\end{align}
Then we square the sum:
\begin{align}
    r_{1} + r_{2}+ \cdots + r_{d} \to (r_{1} + r_{2}+ \cdots + r_{d})^2
\end{align}

One might notice that all of these operations can be done in pure `NumPy`! Let's see what this looks like in `NumPy`:
```python
# Generate `d=ndim` random number: (r1, r2,..., rd)
>>> nums = np.random.rand(ndim)
array([0.26975253, 0.73287346, 0.56619396, 0.36911862, 0.59555464,...])
# Sum them
>>> tsum = np.sum(nums)
2.533493203408602
# Square
>>> tsum**2
6.41858781171758
```
This can be combined into one line as:
```python
>>> np.sum(np.random.rand(ndim))**2
6.41858781171758
```

That was easy! Now, we would like to do this many times. `NumPy` can do this very efficiently. First, let's make an array of `N`x`d` random number:
```python
N = 10
d = 5
>>> rands = np.random.rand(10, 5)
array([[0.26710951, 0.42828954, 0.10725534, 0.90864514],
       [0.89081527, 0.85347667, 0.81367381, 0.53985759],
       [0.11190761, 0.6044705 , 0.10671305, 0.57558664],
       [0.72345535, 0.27989919, 0.42372606, 0.18112357],
       [0.09991332, 0.90279081, 0.65644979, 0.84177427],
       [0.29823545, 0.3388897 , 0.56091701, 0.98475297],
       [0.55350661, 0.47200534, 0.39264102, 0.33956616],
       [0.56339476, 0.92145093, 0.13305375, 0.68434001],
       [0.8425773 , 0.69433766, 0.29323947, 0.32693491],
       [0.43869687, 0.55223718, 0.29408006, 0.41584071]])
```
Now, for each of these rows, we would like to sum up all the entries. This can be done again using `np.sum`. But we need to specify that we would like to sum the columns. This is done by specifying `np.sum(..., axis=1)`. Here, `axis=1` means columns (`axis = 0` means rows.) Let's see what happens:
```python
>>> np.sum(rands, axis=1)
array([1.71129954, 3.09782335, 1.39867779, 1.60820416, 2.50092819,
       2.18279512, 1.75771913, 2.30223945, 2.15708934, 1.70085482])
```
Now, we would like to square each value. We again just use `**`:
```python
>>> np.sum(rands, axis=1)**2
array([2.92854612, 9.59650949, 1.95629956, 2.58632063, 6.25464183,
       4.76459453, 3.08957656, 5.30030649, 4.65303442, 2.8929071 ])
```
Lastly, to get the integral estimate, we can use `np.average`. All together, this looks like:
```python
>>> integral = np.average(np.sum(np.random.rand(10, 5), axis=1)**2)
4.402273672551671
```
We can similarly get the error estimate using `np.std`. Let's see how pure `NumPy` compares to our `integrate_mc` function:

In [521]:
t0 = time.time()
npts = 5000000
vals = np.sum(np.random.rand(npts, 10), axis=1)**2
integal = np.average(vals)
error = np.std(vals) / np.sqrt(npts)
t1 = time.time()

print(f"integral = {integal}, error = {error}, time = {t1 - t0}")

integral = 25.835523342296007, error = 0.0041132507547658095, time = 0.4805889129638672


That was about 50 times faster! Let's try and example:

**Excercise**

Now let's get our hands dirty and see if we can integrate another function. Using the above as a template, use `NumPy` to compute the following integral:
\begin{align}
    I = \prod_{i=1}^{5}\int_{0}^{1}dx_{i}x_{i}^2 = \int_{0}^{1}dx_{1}\cdots\int_{0}^{1}dx_{5}\ x_{1}^{2}\cdots x_{5}^2
\end{align}

Compare you answer with the exact answer, which is $\frac{1}{3^5}$. (**Hint**: NumPy has a product function: `np.prod`.)

In [ ]:
# Your answer here...

## `Multiproccesing`

Basic setup:

Import the module (call it `mp` for short):
```python
import multiprocessing as mp
```
Construct a `Pool`:
```python
num_pools = ...
pool = mp.Pool(num_pools)
```
The above will generate `num_pools` number of proccess which can run independently. In essence, this will be the number of simultaneous `Python` programs that can run at the same time. Typically, I set this the number number of logical cores on the machine I'm working on. To see how many logical cores you have, you can use: `mp.cpu_count()`.

Start running some functions! Say we have some function called `compute` which takes in some arguments. We can start running this function by using:
```python
result = pool.apply_async(compute, args=(...))
```
Here we are running the function `compute` in a seperate python program and storing the result from `compute` in `result`. Next, if we don't want to add anything more to the pool, then we can close the pool:
```python
pool.close()
```
This tells the pool that we won't be adding anything else. Lastly, we will stop the current python program until all the pools have finished their computations using the `join` method:
```python
pool.join()
```
Now, we can get the result once finished!
```python
res = result.get()
```

Let's give this a shot using our Monte-Carlo integrator. We will construct a function which will compute the sum of function values and the sum of function values squared (needed for error estimate). Then, take the total number of evaluations that we need to perform and divide up the work among the different workers in the pool. Once all the workers have finished computing there portion of the sum, we will compute the integral and error.

In [522]:
def generate_fs(f, ndim, npts):
    """
    Integrate a function over a hyper-cube using Monte-Carlo.
    
    Parameters
    ----------
    f: callable
        Function to integrate
    ndim: int
        Number of dimensions of the system
    npts: int
        Number of points to use in integration.
    """
    fsum = 0.0
    f2sum = 0.0
    for k in range(npts):
        # Generate a random vector
        pts = [random.random() for i in range(ndim)]
        # Evaluate the function
        fval = f(pts)
        fsum += fval
        f2sum += fval**2
    return np.array([fsum, f2sum])


def multiproccessing_integrate_mc(f, ndim, npts):
    """
    Integrate a function over a hyper-cube using Monte-Carlo.
    
    Parameters
    ----------
    f: callable
        Function to integrate
    ndim: int
        Number of dimensions of the system
    npts: int
        Number of points to use in integration.
    """
    ncpus = mp.cpu_count()
    pool = mp.Pool(ncpus)
    pts_per_cpu = int(npts / ncpus)
    results = []
    for i in range(ncpus):
        results.append(pool.apply_async(generate_fs, args=(f, ndim, pts_per_cpu)))
    pool.close()
    pool.join()
    
    tup = sum([res.get() for res in results]) / npts
    integral = tup[0]
    avg2 = tup[1]
    std = math.sqrt(avg2 - integral**2)
    error = std / math.sqrt(npts)
    
    return integral, error

In [523]:
t0 = time.time()
npts = 5000000
res = multiproccessing_integrate_mc(f, 10, npts)
t1 = time.time()
print(f"result={res}, time={t1-t0}")

result=(25.83404179977455, 0.0041171372755293085), time=1.7203221321105957


Here we get an answer that is about 5 times faster!

**Excercise**

Below, we have a function called `complicated_function` that takes a number, then runs for 2 seconds and then returns a number for you. Suppose we need to call this function 10 times. That would take about 20 seconds. Use `multiproccessing` to start up a pool of workers to work on these 10 computations simultaneously and print out the results.

In [534]:
def complicated_function(i):
    time.sleep(2)
    return i

In [535]:
t0 = time.time()
results = [complicated_function(i) for i in range(10)]
t1 = time.time()    
print(f'results = {results}, time = {t1 - t0}')

results = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], time = 20.028277158737183


In [536]:
t0 = time.time()
ncpus = mp.cpu_count()
pool = mp.Pool(ncpus)
results = []
for i in range(10):
    results.append(pool.apply_async(complicated_function, args=(i,)))
pool.close()
pool.join()

results = [res.get() for res in results]
    

t1 = time.time()    
print(f'results = {results}, time = {t1 - t0}')

results = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], time = 2.13602614402771


## `Cython`

Here we will describe how to make `Python` lightning fast by using `Cython` https://cython.org/. `Cython` is a hybrid language which looks very similar to `Python` but allows you to add types and call `c/c++` functions. `Cython` code gets compiled into optimized `c` code which is then called from python. 

In jupyter, all of this is very easy. We just need to have `Cython` installed (easier said than done on Windows...) and annotate a cell with a magic function `%%cython`. We can then write `Cython` code in that cell and call the `Cython` code from the rest of our jupyter notebook! Let's take a look at a very simple example.

### Warm-up

We will start with a silly example. Suppose you want to sum up the first `N` integers in `Python`. That is, you want to know the result of:

\begin{align}
S_N = \sum_{i=1}^{N}i
\end{align}

(of course we know the analytic answer, but bare with me.) In `Python` we can write a function which takes `N` as a parameter and runs a quick for loop to sum over the integers between 1 and `N`. This looks like:

In [539]:
def pysum(N):
    int_sum = 0
    for i in range(N + 1):
        int_sum += i
    return int_sum 

Let's see how long this function takes to run with `N` = 50 millon:

In [564]:
%timeit pysum(int(5e7))

2.34 s ± 32.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [569]:
%%cython

def cysum(unsigned long long n):
    # Add types to int_sum and i (note unsigned long long is a 64-bit unsiged integer)
    # with a maximum value of about 10^19
    cdef unsigned long long int_sum = 0
    cdef unsigned long long i
    for i in range(n + 1):
        int_sum += i
    return int_sum
        

In [567]:
%timeit cysum(int(5e7))

157 ns ± 0.95 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Okay...well...that was about 10 millon times faster... Although this example is a bit contrived and this is obviously not the typical speed up that you will find when using `Cython`, it shows that you can achieve MUCH faster code.

Typicaly, you can see anything from 10-1000 times speed up with `Cython`.

### Ising model revisited

Let's revist the Ising model. But let's step our game up. Let's do the 2D Ising mode. Everything is the same as in the 1D Ising model, but with an extra dimension. We will need code to do the following:

- **`initialize_lattice`**: A function to create a random initial state of spins. All spins will be either 1 or -1.
- **`update_lattice_point`**: A function to update the state of a spin using the Metropolis–Hastings algorithm.
- **`update_lattice`**: A function to update all of the spins on the lattice (this is a bit different than what we did in Lecture and in the PSET, but instead of picking a random spin to update, we will update them all for each 'step')

And thats it! For simplicity, we will assume no external magnetic field. Let's write:

In [582]:
def initialize_lattice(N, M):
    """
    Returns an NxM lattice of spins with spin -1 or 1.    
    """
    return np.random.choice([-1, 1], size=(N, M))


def update_lattice_point(lattice, n, m, beta):
    """
    Update the spin located at (n, m) using the Metropolis–Hastings algorithm.

    Parameters
    ----------
    lattice: np.ndarry of shape (N, M)
        Lattice of spins
    n: int
        Row where spin is located.
    m: int
        Column where spin is located.
    beta: float
        Inverse temperature of the system in units of energy.
    """
    N, M = lattice.shape
    # Compute energy needed to flip spin.
    dE = 2 * lattice[n, m] * (
        lattice[(n + 1) % N, m]
        + lattice[(n - 1) % N, m]
        + lattice[n, (m + 1) % M]
        + lattice[n, (m - 1) % M]
    )
    # Decide if we should flip using Metropolis–Hastings
    if np.exp(-dE * beta) > np.random.rand():
        lattice[n, m] *= -1


def update_lattice(lattice, beta):
    """
    Update every point on the lattice once in-place and
    return the lattice.
    
    Paramters
    ----------
    lattice: np.ndarry of shape (N, M)
        Lattice of spins
    beta: float
        Inverse temperature of the system in units of energy.
    """
    N, M = lattice.shape
    for n_offset in range(2):
        for m_offset in range(2):
            for n in range(n_offset, N, 2):
                for m in range(m_offset, M, 2):
                    update_lattice_point(lattice, n, m, beta)
    return lattice

Now, let's write a function to display the lattice. We will use `Pillow` for this example (you can also just use `matplotlib.pyplot.imshow`, but for this case, `Pillow` will be smoother)

In [594]:
def disply_lattice(lattice):
    """
    Generate an image of the lattice in gray scale
    """
    # Return an image of the lattice. Pillow would like values
    # from 0 to 255
    return Image.fromarray(np.uint8((lattice + 1) * 0.5 * 255))

def disply_lattice_sequence(images):
    """
    Display a set of images of the lattice
    """
    def _show(frame=(0, len(images)-1)):
        return disply_lattice(images[frame])
    return interact(_show)

Now, let's initialize the lattice and update every point 50 times.

In [595]:
t0 = time.time()
images = [initialize_lattice(200, 200)]
for i in range(50):
    images.append(update_lattice(images[-1].copy(), 0.7))
t1 = time.time()
print(t1 - t0)

11.547499179840088


Takes about 10 seconds...not so good for making changes to the code...But let's take a look at the sequence of images:

In [593]:
disply_lattice_sequence(images)

interactive(children=(IntSlider(value=25, description='frame', max=50), Output()), _dom_classes=('widget-inter…

<function __main__.disply_lattice_sequence.<locals>._show(frame=(0, 50))>

Very cool!

Now, let's make this ligthning fast! Away we go...

In [598]:
%%cython

# Import Cython's NumPy
import numpy as np
cimport numpy as np
# Import the c version of exp
from libc.math cimport exp
# Import the c random number generator
from libc.stdlib cimport rand
# Import cython for decorators
cimport cython

# The c random number generator generates an integer
# from 0 to RAND_MAX. We need to know this number. 
# The bellow code pulls in this value for the 'limits.h'
# c standard library.
cdef extern from "limits.h":
    int RAND_MAX

@cython.boundscheck(False)
@cython.wraparound(False)
cdef cy_update_lattice_point(np.int64_t[:,:] lattice, int n, int m, double beta):
    # Give types to all the things...
    cdef unsigned int N = lattice.shape[0]
    cdef unsigned int M = lattice.shape[1]

    # Instead of doing (n + 1) % N, we will determine
    # n + 1 and n - 1 ourselves
    cdef int nmax = n + 1
    cdef int nmin = n - 1
    if n == N - 1:
        nmax = 0
    elif n == 0:
        nmin = N - 1
    
    cdef int mmax = m + 1
    cdef int mmin = m - 1
    if m == M - 1:
        mmax = 0
    elif m == 0:
        mmin = M - 1
    
    cdef int dE = 2 * lattice[n, m] * (
          lattice[nmax, m]
        + lattice[nmin, m]
        + lattice[n, mmax]
        + lattice[n, mmin]
    )
    # Equivalent to exp(-dE * beta) > np.random.rand()
    if exp(-dE * beta) * RAND_MAX > rand():
        lattice[n, m] = -lattice[n, m]
    
def cy_update_lattice(np.int64_t[:,:] lattice, double beta):
    cdef int N = lattice.shape[0]
    cdef int M = lattice.shape[1]
    cdef int n, m
    cdef int n_offset, m_offset
    
    for n_offset in range(2):
        for m_offset in range(2):
            for n in range(n_offset, N, 2):
                for m in range(m_offset, M, 2):
                    cy_update_lattice_point(lattice, n, m, beta)
    return np.array(lattice)

Let's see how this new verison compares to the 'pure' `Python` version:

In [604]:
lattice = initialize_lattice(200, 200)
%timeit update_lattice(lattice, 0.4) # = 🐢
%timeit cy_update_lattice(lattice, 0.4) # = 🔥

235 ms ± 6.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
657 µs ± 9.74 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


🏃‍♀️ About 500 times faster!!

In [600]:
images = [initialize_lattice(200, 200)]
for i in range(100):
    images.append(cy_update_lattice(images[-1].copy(), 0.8))

🔥 Going so fast we're burnin' up 🔥

In [603]:
disply_lattice_sequence(images)

interactive(children=(IntSlider(value=50, description='frame'), Output()), _dom_classes=('widget-interact',))

<function __main__.disply_lattice_sequence.<locals>._show(frame=(0, 100))>